In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv("data/train.csv")

In [3]:
df.drop(['ID','society'],axis=1,inplace=True)
#df.dropna(subset=['location'],axis=0,inplace=True)


In [9]:
df.dropna(subset=['location'],axis=0,inplace=True)

In [4]:
# Function to convert total_sqft
#Perch,Sq. Meter,Sq. Yards,Acres,Guntha,Cents and values like 1160-1315
def convert_sqft(sqft_range):
    if '-' in sqft_range:
        start, end = map(float, sqft_range.split('-'))
        return (start + end) / 2
    elif "Sq." in sqft_range : 
        no=str.split(sqft_range,"Sq.")[0] 
        return (no)
    elif "Acres" in sqft_range: 
        no=str.split(sqft_range,"Acres")[0] 
        return (no)
    elif "Guntha" in sqft_range: 
        no=str.split(sqft_range,"Guntha")[0] 
        return (float(no)*1089) 
    elif "Cents" in sqft_range: 
        no=str.split(sqft_range,"Cents")[0] 
        return (float(no)*435.56 ) 
    elif "Perch" in sqft_range: 
        no=str.split(sqft_range,"Perch")[0] 
        return (float(no)*272.25 ) 
    elif "Grounds" in sqft_range: 
        no=str.split(sqft_range,"Grounds")[0] 
        return (float(no)*2400 ) 
    else:
        return float(sqft_range)
    
# Apply the function to the 'total_sqft' column
df['total_sqft'] = df['total_sqft'].apply(convert_sqft)
df['total_sqft'] = df['total_sqft'].astype(float)

In [5]:
#Handle size naa

def handle_size_nan(totalsqrt):
   one_BHK_avg=np.average(df[(df['size']=='1 Bedroom') | (df['size']=='1 BHK')]['total_sqft'])
   two_BHK_avg=np.average(df[(df['size']=='2 Bedroom') | (df['size']=='2 BHK')]['total_sqft'])
   three_BHK_avg=np.average(df[(df['size']=='3 Bedroom') | (df['size']=='3 BHK')]['total_sqft'])
   four_BHK_avg=np.average(df[(df['size']=='4 Bedroom') | (df['size']=='4 BHK')]['total_sqft'])
   if totalsqrt < one_BHK_avg:
        return ("1 BHK")
   elif totalsqrt < two_BHK_avg:
        return("2 BHK")
   elif totalsqrt < three_BHK_avg:
        return("3 BHK")
   elif totalsqrt < four_BHK_avg:
        return("4 BHK")
   else:
        return ("5 BHK")
    

def convert_size(size):
    if 'Bedroom' in size:
       val=str.split(size,"Bedroom")[0].strip()
       return(val+"BHK")
    elif "RK" in size: 
       return("0BHK")
    else:
       return ((str.split(size,"BHK")[0]).strip()+"BHK") 
    

df['size'] =df['size'].fillna(df['total_sqft'].apply(handle_size_nan))
# Apply the function to the 'total_sqft' column to have all values like (1BHK,2BHK ect)
df['size'] = df['size'].apply(convert_size)    


In [7]:
def handle_bathbalcony_nan(size):
    size=str.split(size,"BHK")[0]
    if int(size)< 4:
       return(int(size)-1)
    else:
       return (3)
    return(size)
df['bath'] =df['bath'].fillna(df['size'].apply(handle_bathbalcony_nan))
df['balcony'] =df['balcony'].fillna(df['size'].apply(handle_bathbalcony_nan))

In [ ]:
#plt.figure(figsize=(20,15))
#sns.boxplot(x=df['size'],y=df['total_sqft'])

In [10]:
#Remove outlier
def remove_outlier(size):
    q1=df[df['size']==size]['total_sqft'].quantile(0.25)
    q3=df[df['size']==size]['total_sqft'].quantile(0.75)
    IQR=q3-q1
    lb=q1-(1.5*IQR)
    ub=q3+(1.5*IQR)
    filtered_df = df[(df['size'] == size) & (df['total_sqft'] > ub)]
    df.drop(index=filtered_df.index,axis=0,inplace=True)
remove_outlier('8BHK')
remove_outlier('9BHK')

In [11]:
X=df.drop('price',axis=1)
y=df['price']

In [12]:
numeric_features=['bath', 'balcony','total_sqft']
categorical_features = ['area_type', 'availability', 'location', 'size']
#dfscaler = StandardScaler()
#df[numeric_features] = dfscaler.fit_transform(df[numeric_features])
#df=pd.get_dummies(df, columns=categorical_features, prefix='category')
#X=df.drop('price',axis=1)
#y=df['price']
#model = LinearRegression()
#model.fit(X,y)



In [13]:
def convert_to_float(df):
    df['total_sqft'] = pd.to_numeric(df['total_sqft'], errors='coerce')
    return df

#def drop_feature(df):
    X.drop(['ID','society'], axis=1,inplace=True)
    return df

def drop_NA(df):
    return df.dropna(subset=['location'],axis=0,inplace=True)

def convert_sqft(df):
    return df.assign(total_sqft=df['total_sqft'].apply(convert_sqft).astype(float))


def fillna_size(df):
     return (df.assign(size=df['size'].fillna(handle_size_nan))
    )

def transform_size(df):
    return df.assign(size=df['size'].apply(convert_size))

def fillna_bath(df):
    return df.assign(bath=df['bath'].fillna(df['size'].apply(handle_bathbalcony_nan)))


def fillna_balcony(df):
    return df.assign(bath=df['balcony'].fillna(df['size'].apply(handle_bathbalcony_nan)))


def outlier_removal(df):
    return df.pipe(remove_outlier, '8 BHK').pipe(remove_outlier, '9 BHK')


In [14]:
converttofloat=FunctionTransformer(convert_to_float)
#dropFeature = FunctionTransformer(drop_feature)
dropNA=FunctionTransformer(drop_NA)
convertSqft=FunctionTransformer(convert_sqft)
converttofloat=FunctionTransformer(convert_to_float)
fillnaSize=FunctionTransformer(fillna_size)
transformSize=FunctionTransformer(transform_size)
fillnaBath=FunctionTransformer(fillna_bath)
fillnaBalcony=FunctionTransformer(fillna_balcony)
outlierRemoval=FunctionTransformer(outlier_removal)

In [29]:
numeric_features=['bath', 'balcony','total_sqft']
categorical_features = ['area_type', 'availability', 'location', 'size']
preprocessor = ColumnTransformer(
    transformers=[
        ('dropNA', dropNA, ['location']),
        #('convertSqft', convertSqft, ['total_sqft']),
        ('converttofloat',converttofloat,['total_sqft']),
        ('fillnaSize', fillnaSize, ['size']),
        ('transformSize', transformSize, ['size']),
        ('fillnaBath', fillnaBath, ['bath']),
        ('fillnaBalcony', fillnaBalcony, ['balcony']),
        #('outlierRemoval', outlierRemoval,df),
    ])

In [13]:
model = LinearRegression()

# Define the preprocessing for numerical features: imputation and scaling
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Define the preprocessing for categorical features: imputation and one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create the ColumnTransformer to apply the transformations to the correct columns
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ('cat', categorical_transformer,categorical_features)])

# Create the pipeline
pipeline = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('preprocessor2', preprocessor2),
    ('regressor', model)])

In [14]:
pipeline

Pipeline(steps=[('preprocessor2',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['bath', 'balcony',
                                                   'total_sqft']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['area_type', 'availability',
                                                   'location', 'size'])])),
                ('regressor', LinearRegression())])

In [43]:
#**************LinearRegreesion****************

In [22]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor2',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['bath', 'balcony',
                                                   'total_sqft']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['area_type', 'availability',
                                                   'location', 'size'])])),
                ('regressor', LinearRegression())])

In [37]:
ytrainpred=pipeline.predict(X)

In [40]:
np.sqrt(mean_squared_error(y,ytrainpred))

80.20553014766958

In [ ]:
#Prediction with test

In [32]:
df_test=pd.read_csv("data/test.csv")
df_test.drop(['ID','society'],axis=1,inplace=True)
df_test['total_sqft'] = df_test['total_sqft'].apply(convert_sqft)
df_test['total_sqft'] = df_test['total_sqft'].astype(float)
df_test['size'] =df_test['size'].fillna(df_test['total_sqft'].apply(handle_size_nan))
df_test['size'] = df_test['size'].apply(convert_size)    
df_test['bath'] =df_test['bath'].fillna(df['size'].apply(handle_bathbalcony_nan))
df_test['balcony'] =df_test['balcony'].fillna(df['size'].apply(handle_bathbalcony_nan))

c:\Users\AshwiniKulkarni\anaconda3\Lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)


In [41]:
pipeline.predict(df_test)

array([145.27312757,  71.30651309, 112.42482551, ...,  38.58225745,
       312.62985573,  14.13572577])

In [44]:
#**********RandomForest REgressor***************

In [50]:
RF=RandomForestRegressor()
pipeline_RF = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('preprocessor2', preprocessor2),
    ('regressor', RF)])


In [51]:
pipeline_RF.fit(X,y)

Pipeline(steps=[('preprocessor2',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['bath', 'balcony',
                                                   'total_sqft']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['area_type', 'availability',
                                                   'location', 'size'])])),
                ('regressor', RandomForestRegressor())])

In [52]:
pipeline_RF.predict(df_test)

array([ 31.29095198,  61.7383    , 101.71084667, ...,  45.08159   ,
       381.865     ,  19.0049    ])

In [54]:
ytrainpred=pipeline_RF.predict(X)
np.sqrt(mean_squared_error(y,ytrainpred))

31.2751743465038